<a href="https://colab.research.google.com/github/james330/NLPHW05/blob/main/NLPHW05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 8.9 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486150 sha256=29f6aaacfa67afd77152f4156b7b6dc7d7d697573349b1176da9c2d724f7d2d3
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [ ]:
import json
from opencc import OpenCC
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam # - Works

In [ ]:
# 將json格式，轉為python格式
# {'english':'xxxx' ,'chinese':'xxxxx'}
data = []
datas = []
file = open('./drive/MyDrive/Colab Notebooks/data/translation2019zh_train.json' ,'r')
for f in file:
  data.append(json.loads(f)) #就這樣將json 轉成python了

In [ ]:
# 分割英文數據和中文數據
en_data = []
ch_data = []
for da in data:
  en_data.append(da['english'])
  ch_data.append('\t'+da['chinese']+'\n')

print('英文數據:\n', en_data[:10])
print('\n中文數據:\n', ch_data[:3])

英文數據:
 ['For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'He calls the Green Book, his book of teachings, “the new gospel.', 'And the light breeze moves me to caress her long ear', 'They have the blood of martyrs is the White to flow …', "Finally, the Lakers head to the Motor City to take on a Pistons team that currently owns the Eastern Conference's second best record (1/31). L.", '"The perfect match—my father loves names and Jackie loves money, " sneered Alexander at the wedding. Neither he nor Christina ever got along with their stepmother17.', 'In 2006, Walmart was charged with racism when its recommendation engine paired Planet of the Apes with a documentary about Martin Luther King.', 'The matte as main copper phase in the cleaning. slag was deter- mined by electron probe microscopic analysis.', 'Have you shined your shoes?', 'The Tanning Matrix can be formed by resorcinol and oxazolidine E, and the reactioncharater

In [ ]:
# 分別生成中英文字典，英文
i = 0
en_vocab = set()
for ed in en_data:
  if i == 200:
    break
  else:
    en_vocab = en_vocab|set(''.join(ed))
    i+=1

id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}
print('\n英文字典:\n', en2id)


英文字典:
 {'g': 0, '&': 1, 'j': 2, '4': 3, 'i': 4, 'u': 5, '"': 6, '—': 7, '[': 8, 'O': 9, 'F': 10, 'G': 11, 'p': 12, '/': 13, '7': 14, 'I': 15, '8': 16, '·': 17, 'w': 18, 'A': 19, 'T': 20, 'N': 21, 'q': 22, '“': 23, '2': 24, 'C': 25, 'x': 26, '<': 27, 'P': 28, 'X': 29, 'W': 30, ';': 31, 's': 32, '?': 33, 'f': 34, 'm': 35, 'k': 36, 'M': 37, 'Y': 38, "'": 39, 'z': 40, ' ': 41, 'L': 42, ')': 43, 'V': 44, 'H': 45, '!': 46, '~': 47, 'K': 48, 'e': 49, 'a': 50, 'S': 51, 'b': 52, 'l': 53, 'o': 54, ':': 55, '6': 56, '”': 57, 'Z': 58, 'D': 59, '0': 60, ',': 61, '3': 62, ']': 63, '…': 64, 'E': 65, '-': 66, 'c': 67, '.': 68, '’': 69, 'h': 70, '1': 71, 'd': 72, 'r': 73, 'n': 74, 'y': 75, 't': 76, 'U': 77, '$': 78, '5': 79, '(': 80, '–': 81, '%': 82, 'v': 83, 'B': 84, 'R': 85, '_': 86, 'Q': 87, 'J': 88, '9': 89}


394

In [ ]:
# 分別生成中英文字典，中文
i = 0
ch_vocab = set()
for cd in ch_data:
  if i == 10000:
    break
  else:
    ch_vocab = ch_vocab|set(''.join(cd))
    i+=1

id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}
print('\n中文字典共計\n:', ch2id)


中文字典共計
: {'紧': 0, '艰': 1, '盐': 2, '偷': 3, '罔': 4, '忙': 5, '送': 6, '考': 7, '轻': 8, '步': 9, '疯': 10, '丹': 11, '诙': 12, '荽': 13, '勒': 14, '谈': 15, '旁': 16, '僧': 17, '濡': 18, '呕': 19, '沾': 20, '稿': 21, '欧': 22, '筵': 23, '睡': 24, '墙': 25, '奏': 26, '栋': 27, '兽': 28, 'á': 29, '枪': 30, 'x': 31, '弘': 32, '喳': 33, '浔': 34, '心': 35, '针': 36, '卧': 37, '堪': 38, '癸': 39, '敞': 40, '鹅': 41, '砣': 42, '藉': 43, '拯': 44, '旳': 45, '责': 46, '董': 47, '看': 48, '唉': 49, '刃': 50, '沪': 51, '裹': 52, '艇': 53, 'Y': 54, '首': 55, '鬈': 56, '跃': 57, '蛮': 58, '集': 59, '绵': 60, '端': 61, '狸': 62, '队': 63, '靠': 64, '升': 65, '滩': 66, '誉': 67, '枯': 68, '父': 69, '皆': 70, '旺': 71, '汛': 72, '碧': 73, '嗽': 74, '魅': 75, '诈': 76, '昔': 77, '掰': 78, '矩': 79, '务': 80, '股': 81, '额': 82, '汽': 83, '媒': 84, '”': 85, '继': 86, '含': 87, '觅': 88, '麒': 89, '岁': 90, '仍': 91, '乳': 92, '姆': 93, '洽': 94, '贪': 95, '芳': 96, '膜': 97, '寥': 98, '趟': 99, '肺': 100, '浙': 101, '蜒': 102, '丁': 103, '优': 104, '禾': 105, '猜': 106, '种': 107, 'Ñ': 108, '颗': 109,

In [93]:
i = 0
en_num_data = []
e = []
for line in en_data:
  if i == 10:
    break
  else:
    for en in line:
      e.append(en2id[en])
    en_num_data.append(e)
    e = []
    i+=1

In [100]:
i = 0
ch_num_data = []
c = []
for line in ch_data:
  if i == 10:
    break
  else:
    for ch in line:
      c.append(ch2id[ch])
    ch_num_data.append(c)
    c = []
    i+=1

In [101]:
# ???
i = 0
de_num_data = []
for line in ch_data:
  if i == 10:
    break
  else:
    for ch in line[1:]:
      de_num_data.append(ch2id[ch])
    i+=1

In [103]:
# 獲取輸入輸出端的最大長度
max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

max encoder length: 194
max decoder length: 65


In [105]:
for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

index data:
 [45, 49, 41, 67, 50, 53, 53, 32, 41, 76, 70, 49, 41, 11, 73, 49, 49, 74, 41, 84, 54, 54, 36, 61, 41, 70, 4, 32, 41, 52, 54, 54, 36, 41, 54, 34, 41, 76, 49, 50, 67, 70, 4, 74, 0, 32, 61, 41, 23, 76, 70, 49, 41, 74, 49, 18, 41, 0, 54, 32, 12, 49, 53, 68]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
